<a href="https://colab.research.google.com/github/dasmiq/CS6120-PS3/blob/master/ner_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing a Viterbi Decoder and Evaluation for Sequence Labeling

In this assignment, you will build a Viterbi decoder for an LSTM named-entity recognition model. As we mentioned in class, recurrent and bidirectional recurrent neural networks, of which LSTMs are the most common examples, can be used to perform sequence labeling. Although these models encode information from the surrounding words in order to make predictions, there are no "hard" constraints on what tags can appear where.

There hard constraints are particularly important for tasks that label spans of more than one token. The most common example of a span-labeling task is named-entity recognition (NER). As described in Eisenstein, Jurafksy & Martin, and other texts, the goal of NER is to label spans of one or more words as _mentions_ of an _entity_, such as a person, location, organization, etc.

The most common approach to NER is to reduce it to a sequence-labeling task, where each token in the input is labeled either with an `O`, if it is "outside" any named-entity span, or with `B-TYPE`, if it is the first token in an entity of type `TYPE`, or with `I-TYPE`, if it is the second or later token in an entity of type `TYPE`. Distinguishing between the first and later tokens of an entity allow us to identify distinct entity spans even when they are adjacent.

Common values of `TYPE` include `PER` for person, `LOC` for location, `DATE` for date, and so on. In the dataset we load below, there are 17 distinct types.

The span-labeling scheme just described implies that the labels on tokens must obey certain constraints: the tag `I-PER` must follow either `B-PER` or another `I-PER`. It cannot follow `O`, `B-LOC`, or `I-LOC`, i.e., a tag for a different entity type. By themselves, LSTMs or bidirectional LSTMs cannot directly enforce these constraints. This is one reason why conditional random fields (CRFs), which _can_ enforce these constraints, are often layered on top of these recurrent models.

In this assignment, you will implement the simplest possible CRF: a CRF so simple that it does not require any training. Rather, it will assign weight 1 to any sequence of tags that obeys the constraints and weight 0 to any sequence of tags that violates them. The inputs to the CRF, which are analogous to the emission probabilities in an HMM, will come from an LSTM.

But first, in order to test your decoder, you will also implement some functions to evaluate the output of an NER system according to two metrics:
1. You will count the number of _violations_ of the NER label constraints, i.e., how many times `I-TYPE` follows `O` or a tag of a different type or occurs at the beginning of a sentence. This number will be greater than 0 in the raw LSTM output, but should be 0 for your CRF output.
1. You will compute the _span-level_ precision, recall, and F1 of NER output. Although the baseline LSTM was trained to achieve high _token-level_ accuracy, this metric can be misleadingly high, since so many tokens are correctly labeled `O`. In other words, what proportion of spans predicted by the model line up exactly with spans in the gold standard, and what proportion of spans in the gold standard were predicted by the model? Define _span_ as a sequence of tags that starts with a `B-TYPE` followed by zero or more `I-TYPE` tags. Sequences solely of `I-TYPE` tags don't count as spans.For more, see the original task definition: https://www.aclweb.org/anthology/W03-0419/.

We start with loading some code and data and the describe your tasks in more detail.

## Set Up Dependencies and Definitions

In [1]:
!pip install --upgrade spacy==2.1.0 allennlp==0.9.0
import spacy
print(spacy.__version__)

Requirement already up-to-date: spacy==2.1.0 in c:\tools\anaconda3\lib\site-packages (2.1.0)
Requirement already up-to-date: allennlp==0.9.0 in c:\tools\anaconda3\lib\site-packages (0.9.0)
2.1.0


In [194]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
import operator
import math
import re
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data.dataset_readers import conll2003

torch.manual_seed(1)

In [3]:
class LstmTagger(Model):
  def __init__(self,
               word_embeddings: TextFieldEmbedder,
               encoder: Seq2SeqEncoder,
               vocab: Vocabulary) -> None:
    super().__init__(vocab)
    self.word_embeddings = word_embeddings
    self.encoder = encoder
    self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))
    self.accuracy = CategoricalAccuracy()

  def forward(self,
              tokens: Dict[str, torch.Tensor],
              metadata,
              tags: torch.Tensor = None) -> Dict[str, torch.Tensor]:
    mask = get_text_field_mask(tokens)
    embeddings = self.word_embeddings(tokens)
    encoder_out = self.encoder(embeddings, mask)
    tag_logits = self.hidden2tag(encoder_out)
    output = {"tag_logits": tag_logits}
    if tags is not None:
      self.accuracy(tag_logits, tags, mask)
      output["loss"] = sequence_cross_entropy_with_logits(tag_logits, tags, mask)

    return output

  def get_metrics(self, reset: bool = False) -> Dict[str, float]:
    return {"accuracy": self.accuracy.get_metric(reset)}

## Import Data

In [4]:
reader = conll2003.Conll2003DatasetReader()
train_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.train.ner.sample'))
validation_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.development.ner.sample'))

vocab = Vocabulary.from_instances(train_dataset + validation_dataset)

562it [00:00, 13704.88it/s]
23it [00:00, 2090.38it/s]
100%|██████████| 585/585 [00:00<00:00, 68815.01it/s]


## Define and Train Model

In [5]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, bidirectional=False, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, eps=1e-8)
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=100,
                  cuda_device=cuda_device)
trainer.train()

accuracy: 0.8442, loss: 0.9153 ||: 100%|██████████| 281/281 [00:02<00:00, 94.02it/s]
accuracy: 0.7878, loss: 1.2134 ||: 100%|██████████| 12/12 [00:00<00:00, 275.81it/s]
accuracy: 0.8442, loss: 0.7302 ||: 100%|██████████| 281/281 [00:01<00:00, 186.72it/s]
accuracy: 0.7878, loss: 1.2023 ||: 100%|██████████| 12/12 [00:00<00:00, 311.63it/s]
accuracy: 0.8442, loss: 0.7162 ||: 100%|██████████| 281/281 [00:01<00:00, 181.20it/s]
accuracy: 0.7878, loss: 1.1760 ||: 100%|██████████| 12/12 [00:00<00:00, 333.28it/s]
accuracy: 0.8442, loss: 0.7071 ||: 100%|██████████| 281/281 [00:01<00:00, 185.71it/s]
accuracy: 0.7878, loss: 1.1728 ||: 100%|██████████| 12/12 [00:00<00:00, 333.27it/s]
accuracy: 0.8442, loss: 0.6984 ||: 100%|██████████| 281/281 [00:01<00:00, 198.13it/s]
accuracy: 0.7878, loss: 1.1806 ||: 100%|██████████| 12/12 [00:00<00:00, 347.77it/s]
accuracy: 0.8442, loss: 0.6913 ||: 100%|██████████| 281/281 [00:01<00:00, 173.80it/s]
accuracy: 0.7878, loss: 1.1707 ||: 100%|██████████| 12/12 [00:00<

{'best_epoch': 99,
 'peak_cpu_memory_MB': 0,
 'peak_gpu_0_memory_MB': 5435,
 'training_duration': '0:02:34.897990',
 'training_start_epoch': 0,
 'training_epochs': 99,
 'epoch': 99,
 'training_accuracy': 0.9202414998411185,
 'training_loss': 0.21481472212681216,
 'training_cpu_memory_MB': 0.0,
 'training_gpu_0_memory_MB': 5277,
 'validation_accuracy': 0.8653061224489796,
 'validation_loss': 0.42215411698756117,
 'best_validation_accuracy': 0.8653061224489796,
 'best_validation_loss': 0.42215411698756117}

## Evaluation

The simple code below creators a `predictor` object that applies the model to an input example and then loops over the examples in the validation set, printing out the input token, gold-standard output, and model output. You can see from these methods how to access data and model outputs for evaluation.

In [35]:
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)

def tag_sentence(s):
  tag_ids = np.argmax(predictor.predict_instance(s)['tag_logits'], axis=-1)
  print(tag_ids)
  fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
  return list(fields)

baseline_output = [tag_sentence(i) for i in validation_dataset]

[0 0 0 0 0 0 0 6 2 0 0 0 0 0 4 0 0 0 0 0 0 0 0 5 5 5 0]
[0 3 0 6 2 2 0 0 0 5 5 5 0 0 0 0 0 0 0 0 0 3 2 2 0]
[0 0 0 4 0 0 0 0 5 5 5 0]
[0 0 0 0 0 0 0 0 3 2 0 0]
[0 0 0 0 0 0 0 0 3 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 0  0  0  0  0  4 13  2  0  0  0  0]
[0 5 5 5 0 0 0 0 0 0 0 0 6 5 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0 0 0 0 4 0]
[0 0 6 2 0 0 0 1 0 1 0 0 0 0 0 0 2 2 0 0 0]
[0 8 2 0 3 0 0 0 0 0 0 7 0 0 0 1 0 0 0 0 0 0 0 0 0 0 4 2 2 0 0 3 2 0 0 2 2
 2 0 0 0 0 0 0 6 3 0 0 3 2 0 0 0 0 6 2 0 0 0 0 0 6 2 3 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 7 0 0 0 0 0 0 0 0 0 0 0 0 0 6 2 0]
[0 0 0 0 0 0 0 0 6 2 0 0 0 0 0 0 0 0 0 0 0 0 6 2 3 2 0]
[0 0 0 0 0 0 0 0 0 0 0 0 6 2 0 0 0 0 0 0 0 0 0 0 0]
[0 3 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 4 3 0 3 0 0 0 0]
[0 0 0 3 0 0 7 

Now, you can implement two evaluation functions: `violations` and `span_stats`.

In [243]:
# TODO: count the number of NER label violations,
# such as O followed by I-TYPE or B-TYPE followed by
# I-OTHER_TYPE
# Take tagger output as input
def violations(tagged):
    count = 0
    for setence in tagged:
        prev_tag = None
        for tagging in setence:
            current_tag = tagging[2]
            if not prev_tag:
                prev_tag = tagging[2]
            else:
                if current_tag == "O":
                    prev_tag = current_tag
                elif current_tag[0] == "I" and current_tag[1:] == prev_tag[1:]:
                    prev_tag = current_tag
                elif current_tag[0] == "B":
                    prev_tag = current_tag
                else:
                    count += 1
    return count

# TODO: return the span-level precision, recall, and F1
# Only count valid spans that start with a B tag,
# followed by zero or more I tags of the same type.
# Take tagger output as input
def span_stats(tagged):
    true_positive = 0
    precision_denom = 0
    recall_denom = 0
    for sentence in tagged:
        golden = [tag[1] for tag in sentence]
        golden_spans = []
        span = False
        spanstring = ""
        for tag in golden:
            if tag != "O" and not span:
                span = True;
            if tag == "O" and span:
                span = False;
                spanstring += tag
                golden_spans.append(spanstring)
                spanstring = ""
            if span:
                spanstring += tag

        predicted = [tag[2] for tag in sentence]
        predicted_spans = []
        span = False
        spanstring = ""
        for tag in predicted:
            if tag[0] == "B" and not span:
                span = True;
            if tag == "O" and span:
                span = False;
                spanstring += tag
                predicted_spans.append(spanstring)
                spanstring = ""
            if span:
                spanstring += tag

        recall_denom += len(golden_spans)
        precision_denom += len(predicted_spans)
        true_positive += len([tag for tag in predicted_spans if tag in golden_spans])
    

    precision = true_positive/precision_denom
    recall = true_positive/recall_denom
    f1 = 2*(precision*recall/(precision+recall))

    return {'precision': precision,
          'recall': recall,
          'f1': f1}

## You can check how many violations are made by the model output in predictor.
print(violations(baseline_output))

60


{'precision': 0.2857142857142857,
 'recall': 0.19047619047619047,
 'f1': 0.22857142857142854}

## Decoding

Now you can finally implement the simple Viterbi decoder. The `predictor` object, when applied to an input sentence, first calculates the scores for each possible output tag for each token. See the line `predictor.predict_instance(i)['tag_logits']` in the code above.

Then, you will construct a transition matrix. You can use the code below to get a list of the tags the model knows about. For a set of K tags, construct a K-by-K matrix with a log(1)=0 when a transition between a given tag pair is valid and a log(0)=-infinity otherwise.

Finally, implement a Viterbi decoder that takes the predictor object and a dataset object and outputs tagged data, just like the `tag_sentence` function above. It should use the Viterbi algorithm with the (max, plus) semiring. You'll be working with sums of log probabilities instead of products of probabilties.

Run your `violations` function on the output of this decoder to make sure that there are no invalid tag transitions. Also, compare the span-level metrics on `baseline_output` and your new output using your `span_stats` function.

In [204]:
# This code show how to map from output vector components to labels
print(vocab.get_index_to_token_vocabulary('labels'))
transition_matrix = np.zeros([len(vocab.get_index_to_token_vocabulary('labels')), len(vocab.get_index_to_token_vocabulary('labels'))])
for y_ in range(len(vocab.get_index_to_token_vocabulary('labels'))):
    for x_ in range(len(vocab.get_index_to_token_vocabulary('labels'))):
        if vocab.get_index_to_token_vocabulary('labels')[x_] == "O":
            transition_matrix[x_][y_] = math.log(1)
        elif vocab.get_index_to_token_vocabulary('labels')[x_][0] == "B":
            transition_matrix[x_][y_] = math.log(1)
        elif vocab.get_index_to_token_vocabulary('labels')[x_][0] == "I" and vocab.get_index_to_token_vocabulary('labels')[x_][1:] == vocab.get_index_to_token_vocabulary('labels')[y_][1:]:
            transition_matrix[x_][y_] = math.log(1)
        else:
            transition_matrix[x_][y_] = -math.inf
               

{0: 'O', 1: 'B-GPE', 2: 'I-ORG', 3: 'I-DATE', 4: 'B-CARDINAL', 5: 'I-EVENT', 6: 'B-PERSON', 7: 'B-NORP', 8: 'B-DATE', 9: 'B-ORG', 10: 'B-LOC', 11: 'I-LOC', 12: 'I-FAC', 13: 'I-PERSON', 14: 'I-GPE', 15: 'I-CARDINAL', 16: 'B-EVENT', 17: 'I-TIME', 18: 'I-WORK_OF_ART', 19: 'B-ORDINAL', 20: 'B-FAC', 21: 'B-TIME', 22: 'I-LAW', 23: 'I-QUANTITY', 24: 'I-NORP', 25: 'I-MONEY', 26: 'B-MONEY', 27: 'B-WORK_OF_ART', 28: 'B-QUANTITY', 29: 'B-LAW', 30: 'B-PRODUCT', 31: 'I-PRODUCT', 32: 'B-PERCENT', 33: 'I-PERCENT'}
[[  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [-inf -inf   0. ... -inf -inf -inf]
 ...
 [-inf -inf -inf ...   0. -inf -inf]
 [  0.   0.   0. ...   0.   0.   0.]
 [-inf -inf -inf ... -inf   0.   0.]]


In [233]:
def viterbi_helper (predictor, dataset):
    # code modified from https://stackoverflow.com/questions/9729968/python-implementation-of-viterbi-algorithm/9730083
    conditional_probabilities = np.array([np.array(xi) for xi in predictor.predict_instance(dataset)['tag_logits']]).T
    transition_probabilities = transition_matrix.T
    num_samples = conditional_probabilities.shape[1]
    num_states = transition_probabilities.shape[0] # number of states

    c = np.zeros(num_samples) #scale factors (necessary to prevent underflow)
    viterbi = np.zeros((num_states,num_samples)) # initialise viterbi table
    best_path_table = np.zeros((num_states,num_samples)) # initialise the best path table
    best_path = np.zeros(num_samples).astype(np.int32) # this will be your output

    # B- appoint initial values for viterbi and best path (bp) tables - Eq (32a-32b)
    viterbi[:,0] = conditional_probabilities[:,0]
    c[0] = 1.0/np.sum(viterbi[:,0])
    viterbi[:,0] = c[0] + viterbi[:,0] # apply the scaling factor

    # C- Do the iterations for viterbi and psi for time>0 until T
    for t in range(1, num_samples): # loop through time
        for s in range (0,num_states): # loop through the states @(t-1)
            trans_p = viterbi[:, t-1] + transition_probabilities[:,s] # transition probs of each state transitioning
            best_path_table[s,t], viterbi[s,t] = max(enumerate(trans_p), key=operator.itemgetter(1))
            viterbi[s,t] = viterbi[s,t] + conditional_probabilities[s][t]

        c[t] = 1.0/np.sum(viterbi[:,t]) # scaling factor
        viterbi[:,t] = c[t] * viterbi[:,t]

    ## D - Back-tracking
    best_path[num_samples-1] =  viterbi[:,num_samples-1].argmax() # last state
    for t in range(num_samples-1,0,-1): # states of (last-1)th to 0th time step
        best_path[t-1] = best_path_table[best_path[t],t]
    return best_path

In [240]:
 def viterbi(predictor, dataset):
    fields = zip(dataset['tokens'], dataset['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in viterbi_helper(predictor, dataset)])
    return list(fields)


In [241]:
viterbi_output = [viterbi(predictor, i) for i in validation_dataset]

In [244]:
violations(viterbi_output)

0

In [245]:
span_stats(viterbi_output)

{'precision': 0.4883720930232558, 'recall': 0.5, 'f1': 0.4941176470588235}

In [246]:
span_stats(baseline_output)

{'precision': 0.2857142857142857,
 'recall': 0.19047619047619047,
 'f1': 0.22857142857142854}